In [1]:
import cv2
import numpy as np
import os
from tqdm import tqdm

In [2]:

def augment_image(img):
    augmented_images = []

    # 1. Rotate 15 degrees
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w//2, h//2), 15, 1)
    rotated = cv2.warpAffine(img, M, (w, h))
    augmented_images.append(rotated)

    # 2. Horizontal flip
    flipped = cv2.flip(img, 1)
    augmented_images.append(flipped)

    # 3. Change brightness
    brightness = cv2.convertScaleAbs(img, alpha=1, beta=40)
    augmented_images.append(brightness)

    # 4. Add Gaussian noise
    noise = img + np.random.normal(0, 12, img.shape)
    noise = np.clip(noise, 0, 255).astype(np.uint8)
    augmented_images.append(noise)

    # 5. Zoom (cropping and resizing)
    zoom = img[20:220, 20:220]
    zoom = cv2.resize(zoom, (w, h))
    augmented_images.append(zoom)

    return augmented_images


In [3]:
def augment_dataset(input_path, output_path):
    for category in os.listdir(input_path):
        input_folder = os.path.join(input_path, category)
        output_folder = os.path.join(output_path, category)

        os.makedirs(output_folder, exist_ok=True)

        print(f"\nProcessing {category}...")

        for img_name in tqdm(os.listdir(input_folder)):
            img_path = os.path.join(input_folder, img_name)
            img = cv2.imread(img_path)

            if img is None:
                continue

            # Save original
            cv2.imwrite(os.path.join(output_folder, img_name), img)

            # Generate augmented images
            augmented = augment_image(img)

            # Save each one
            for i, aug in enumerate(augmented):
                new_name = img_name.replace(".jpg", f"_aug{i}.jpg")
                cv2.imwrite(os.path.join(output_folder, new_name), aug)


In [6]:
input_folder = r"C:\Users\pc\PycharmProjects\Material Stream Identification System\dataset"
output_folder = r"C:\Users\pc\PycharmProjects\Material Stream Identification System\augmented_dataset"

augment_dataset(input_folder, output_folder)



Processing cardboard...


100%|██████████| 259/259 [00:10<00:00, 25.30it/s]



Processing glass...


100%|██████████| 401/401 [00:16<00:00, 24.88it/s]



Processing metal...


...

100%|██████████| 476/476 [00:21<00:00, 22.26it/s]



Processing plastic...


100%|██████████| 386/386 [00:16<00:00, 24.12it/s]



Processing trash...


100%|██████████| 110/110 [00:04<00:00, 23.54it/s]


In [7]:
def count_images_in_folder(folder_path):
    valid_ext = (".jpg", ".jpeg", ".png", ".bmp")
    stats = {}

    for class_name in os.listdir(folder_path):
        class_path = os.path.join(folder_path, class_name)
        if os.path.isdir(class_path):
            img_count = sum(
                file.lower().endswith(valid_ext)
                for file in os.listdir(class_path)
            )
            stats[class_name] = img_count

    total = sum(stats.values())
    return stats, total


if __name__ == "__main__":
    input_folder = r"C:\Users\pc\PycharmProjects\Material Stream Identification System\dataset"
    output_folder = r"C:\Users\pc\PycharmProjects\Material Stream Identification System\augmented_dataset"

    print("🔹 Counting images in dataset...")
    stats_before, total_before = count_images_in_folder(input_folder)
    print("\nImages Before Augmentation:")
    for k, v in stats_before.items():
        print(f"{k}: {v}")
    print(f"Total: {total_before}")

    if os.path.exists(output_folder):
        print("\n🔹 Counting images in augmented dataset...")
        stats_after, total_after = count_images_in_folder(output_folder)
        print("\nImages After Augmentation:")
        for k, v in stats_after.items():
            print(f"{k}: {v}")
        print(f"Total: {total_after}")
    else:
        print("\n⚠️ No augmented dataset found yet!")




🔹 Counting images in dataset...

Images Before Augmentation:
cardboard: 259
glass: 401
metal: 328
paper: 476
plastic: 386
trash: 110
Total: 1960

🔹 Counting images in augmented dataset...

Images After Augmentation:
cardboard: 1482
glass: 2310
metal: 1890
paper: 2694
plastic: 2178
trash: 636
Total: 11190
